In [24]:
from pyswip.prolog import Prolog
from pyswip.easy import *
import pandas as pd

prolog = Prolog()

file = ""

# prolog.assertz("greater_than(X, Y) :- X @> Y")

file += "less_or_equal(X,Y) :- X @=< Y.\n"

# at least one element is equal
# prolog.assertz("at_least_one_in_second([],[]):- false")
# prolog.assertz("at_least_one_in_second([H|_],List):- is_member(H,List) , !")
# prolog.assertz("at_least_one_in_second([H|T],List):- \+is_member(H,List) , compare1(T,List)")

# all elements on the first list is on the second list
file += "all_first_in_second(List1, List2) :- forall(member(Element,List1), member(Element,List2)).\n"

recipes_df = pd.read_csv("recipes.csv")

def clean_alt_list(list_):
    list_ = list_.replace('\"', '\'')
    return list_

recipes_df["title"] = recipes_df["title"].apply(clean_alt_list)

#recipes_df["ingredients_ids"] = recipes_df["ingredients_ids"].apply(eval)
#recipes_df["ingredients_names"] = recipes_df["ingredients_names"].apply(eval)

# register each recipe with two different types of search
for index, row in recipes_df.iterrows():
    """
    % search with ingredients
    recipe(id, title, url) :- 
        search(ingredients), 
        available_ingredients(List_ingredients),
        all_first_in_second(Ingredients, List_ingredients), # all recipe ingredients need to be available
        max_time(Time),
        less_or_equal(Recipe_time, Time),
        max_calories(Calories),
        less_or_equal(Recipe_calories, Calories),
        cuisine(Cuisine),
        member(Cuisine, List_cuisines),
        dishType(Dish),
        member(Dish, List_dishes).
    """
    file += "recipe(" + str(row["id"]) + ",\"" + str(row["title"]) + "\",\"" + str(row["url"]) + "\"):-"
    
    file += "search(ingredient),"
    file += "available_ingredients(List_ingredients),"
    file += "all_first_in_second("+str(row["ingredients_names"])+", List_ingredients),"
    file += "max_time(Time),"
    file += "less_or_equal("+str(row["minutes"])+", Time),"
    file += "max_calories(Calories),"
    file += "less_or_equal("+str(row["calories"])+", Calories),"
    file += "cuisine(Cuisine),"
    file += "member(Cuisine, "+str(row["cuisines"])+"),"
    file += "dishType(Dish),"
    file += "member(Dish, "+str(row["dishTypes"])+").\n\n"
                    
    
    """
    % search with type
    recipe(id, title, url) :- 
        search(type), 
        max_time(Time),
        less_or_equal(Recipe_time, Time),
        max_calories(Calories),
        less_or_equal(Recipe_calories, Calories),
        cuisine(Cuisine),
        member(Cuisine, List_cuisines),
        dishType(Dish),
        member(Dish, List_dishes),
        diet(Diet),
        member(Diet, List_diets),
        max_price(Price),
        less_or_equal(Recipe_price, Price).
    """
    
    file += "recipe(" + str(row["id"]) + ",\"" + str(row["title"]) + "\",\"" + str(row["url"]) + "\"):-"
    file += "search(type),"
    file += "max_time(Time),"
    file += "less_or_equal("+str(row["minutes"])+", Time),"
    file += "max_calories(Calories),"
    file += "less_or_equal("+str(row["calories"])+", Calories),"
    file += "cuisine(Cuisine),"
    file += "member(Cuisine, "+str(row["cuisines"])+"),"
    file += "dishType(Dish),"
    file += "member(Dish, "+str(row["dishTypes"])+"),"
    file += "diet(Diet),"
    file += "member(Diet, "+str(row["diets"])+"),"
    file += "max_price(Price),"
    file += "less_or_equal("+str(row["price"])+", Price).\n\n"
    

In [25]:
# askables
file += "search(X) :- ask(search, X).\n"
file += "available_ingredients(X) :- ask(available_ingredients, X).\n"
file += "max_time(X) :- ask(max_time, X).\n"
file += "max_calories(X) :- ask(max_calories, X).\n"
file += "cuisine(X) :- ask(cuisine, X).\n"
file += "diet(X) :- ask(diet, X).\n"
file += "max_price(X) :- ask(max_price, X).\n"
file += "dishType(X) :- ask(dishType, X).\n"

In [26]:
text_file = open("KB.pl", "w", encoding="utf-8")
n = text_file.write(file)
text_file.close()

In [2]:
recipes_df.head(5)

,Unnamed: 0,id,title,url,minutes,ingredients_ids,ingredients_names,cuisines,dishTypes,calories,diets
0,0,633921,Balsamic & Honey Glazed Salmon with Lemony Asp...,https://spoonacular.com/balsamic-honey-glazed-...,60,"[2048, 11011, 2069, 2017, 11215, 4517, 1012042...","['apple cider vinegar', 'asparagus', 'balsamic...",[],"['lunch', 'main course', 'main dish', 'dinner']",362,"['dairyFree', 'glutenFree']"
1,1,640677,Creamy Porcini Mushroom Polenta,https://spoonacular.com/creamy-porcini-mushroo...,45,"[1001, 10011268, 1053, 1033, 10035137, 2047, 1...","['butter', 'dried porcini mushrooms', 'heavy c...",[],['side dish'],263,['glutenFree']
2,2,636365,Brussels Sprouts with Bacon and Shallots,https://spoonacular.com/brussels-sprouts-with-...,45,"[10123, 11098, 9152, 1102047, 11677]","['bacon', 'brussels sprouts', 'lemon juice', '...",[],['side dish'],124,"['dairyFree', 'glutenFree']"
3,3,646673,Herb Roasted Chicken,https://spoonacular.com/herb-roasted-chicken-6...,240,"[2004, 19334, 1001, 2003, 2036, 2042, 2018, 14...","['bay leaves', 'brown sugar', 'butter', 'dried...",[],"['lunch', 'main course', 'main dish', 'dinner']",425,"['glutenFree', 'lowFodmap']"
4,4,663824,Trinidadian Chicken Potato Curry,https://spoonacular.com/trinidadian-chicken-po...,45,"[1005091, 10511282, 11215, 2049, 10011819, 202...","['bone in chicken thighs', 'yellow onion', 'ga...","['Indian', 'Asian']","['lunch', 'main course', 'main dish', 'dinner']",554,"['dairyFree', 'glutenFree']"


In [4]:
ingredients

['apple cider vinegar',
 'asparagus',
 'balsamic vinegar',
 'dried dill',
 'garlic',
 'grapeseed oil',
 'herbs de provence',
 'honey',
 'juice of lemon',
 'lemon zest',
 'salmon fillet',
 'salt and pepper',
 'vegetable stock']

In [2]:
len(ingredients)

1078

In [3]:
# ['black pepper', 'brussels sprouts', 'dijon mustard', 'honey', 'olive oil', 'red wine vinegar', 'walnuts']
# [1002030, 11098, 1002046, 19296, 4053, 1022068, 12155]
prolog.assertz('ingredient(1002030, "black pepper")')
prolog.assertz('ingredient(11098, "brussels sprouts")')
prolog.assertz('ingredient(1002046, "dijon mustard")')
prolog.assertz('ingredient(19296, "honey")')
prolog.assertz('ingredient(4053, "olive oil")')
prolog.assertz('ingredient(1022068, "red wine vinegar")')
prolog.assertz('ingredient(12155, "walnuts")')
prolog.assertz('ingredient(16015,"black beans")')
prolog.assertz('ingredient(98849,"fire roasted tomatoes")')
prolog.assertz('ingredient(11282,"onion")')
prolog.assertz('ingredient(10011821,"orange bell pepper")')
prolog.assertz('ingredient(6599,"enchilada sauce")')
prolog.assertz('ingredient(6016,"cream of chicken soup")')
prolog.assertz('ingredient(1077,"milk")')
prolog.assertz('ingredient(11174,"cream corn")')
prolog.assertz('ingredient(31015,"green chiles")')
prolog.assertz('ingredient(11979,"jalapeno")')
prolog.assertz('ingredient(1055062,"skinless boneless chicken breast")')
prolog.assertz('ingredient(1001025,"monterey jack cheese")')
prolog.assertz('ingredient(1001009,"shredded cheddar cheese")')
prolog.assertz('ingredient(1056,"sour cream")')
prolog.assertz('ingredient(19056,"tortilla chips")')

prolog.assertz('time(200)')
prolog.assertz('calories(3000)')

prolog.assertz('type_search(X) :- ask(type_search, X)')

In [4]:
for soln in prolog.query("recipe(X,Y,Z,K,L)"):
    print(soln["Y"])

PrologError: Caused by: 'recipe(X,Y,Z,K,L)'. Returned: 'error(existence_error(procedure, /(ask, 2)), context(/(type_search, 1), _1234))'.

In [5]:
retractall = Functor("retractall")
ingredient = Functor("ingredient",2)
call(retractall(ingredient))

1